In [65]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import shutil

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def loadJson(jsonFile):
    fin = open(jsonFile)
    data = json.load(fin)
    lumis = pd.DataFrame(data['data'], columns=data['columns'])
    return lumis

goodRunsName = '/eos/experiment/milliqan/Configs/goodRunsList.json'
lumisName = '/eos/experiment/milliqan/Configs/mqLumis.json'
shutil.copy(goodRunsName, 'goodRunsList.json')
shutil.copy(lumisName, 'mqLumis.json')

goodRuns = loadJson('goodRunsList.json')
lumis = loadJson('mqLumis.json')

In [4]:
#get list of files to look at
files = []

beam = True

dataDir = '/store/user/milliqan/trees/v34/1100/'

for ifile, filename in enumerate(os.listdir(dataDir)):
    #if len(files) > 10: break
    if ifile > 100: break
    
    if not filename.endswith('root'): continue
    
    runNum = int(filename.split('Run')[1].split('.')[0])
    fileNum = int(filename.split('.')[1].split('_')[0])
        
    goodRun = goodRuns['goodRunTight'].loc[(goodRuns['run'] == runNum) & (goodRuns['file'] == fileNum)]
    beamOn = lumis['beam'].loc[(lumis['run'] == runNum) & (lumis['file'] == fileNum)]

    if len(goodRun) == 0 or len(beamOn) == 0: continue

    print(filename, runNum, fileNum, beamOn.item(), goodRun.item())
    
    if beam:
        if goodRun.item() and beamOn.item(): files.append(dataDir+filename)
    else:
        if goodRun.item() and not beamOn.item(): files.append(dataDir+filename)


files, len(files)

MilliQan_Run1100.1_v34.root 1100 1 1.0 False
MilliQan_Run1101.1_v34.root 1101 1 1.0 True
MilliQan_Run1101.2_v34.root 1101 2 1.0 True
MilliQan_Run1101.3_v34.root 1101 3 1.0 True
MilliQan_Run1101.4_v34.root 1101 4 1.0 True
MilliQan_Run1102.2_v34.root 1102 2 1.0 False
MilliQan_Run1103.10_v34.root 1103 10 1.0 False
MilliQan_Run1103.11_v34.root 1103 11 1.0 False
MilliQan_Run1103.13_v34.root 1103 13 1.0 False
MilliQan_Run1103.14_v34.root 1103 14 1.0 False
MilliQan_Run1103.15_v34.root 1103 15 1.0 False
MilliQan_Run1103.16_v34.root 1103 16 1.0 False
MilliQan_Run1103.17_v34.root 1103 17 1.0 False
MilliQan_Run1103.18_v34.root 1103 18 1.0 False
MilliQan_Run1103.19_v34.root 1103 19 1.0 False
MilliQan_Run1103.20_v34.root 1103 20 1.0 False
MilliQan_Run1103.21_v34.root 1103 21 1.0 False
MilliQan_Run1103.2_v34.root 1103 2 1.0 False
MilliQan_Run1103.3_v34.root 1103 3 1.0 False
MilliQan_Run1103.5_v34.root 1103 5 1.0 False
MilliQan_Run1103.6_v34.root 1103 6 1.0 False
MilliQan_Run1103.7_v34.root 1103 7 1.

(['/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.1_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.2_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.3_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.4_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1000_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1001_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1002_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1003_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1004_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1005_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1006_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1007_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1008_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1009_v34.root',
  '/

In [111]:
@mqCut
def pulseTime(self):
    events = self.events
    straightPath = self.events['straightLineCutPulse']

    timeToUse = 'timeFit_module_calibrated_corrected'

    self.events['straightPathL0Time'] = events[timeToUse][events['layer0'] & events['straightLineCutPulse']]
    self.events['straightPathL1Time'] = events[timeToUse][events['layer1'] & events['straightLineCutPulse']]
    self.events['straightPathL2Time'] = events[timeToUse][events['layer2'] & events['straightLineCutPulse']]
    self.events['straightPathL3Time'] = events[timeToUse][events['layer3'] & events['straightLineCutPulse']]

    height0 = ak.max(self.events['height'][events['layer0'] & events['straightLineCutPulse']], axis=1)
    height1 = ak.max(self.events['height'][events['layer1'] & events['straightLineCutPulse']], axis=1)
    height2 = ak.max(self.events['height'][events['layer2'] & events['straightLineCutPulse']], axis=1)
    height3 = ak.max(self.events['height'][events['layer3'] & events['straightLineCutPulse']], axis=1)

    mask0 = (self.events['height'][events['layer0'] & events['straightLineCutPulse']] == height0)
    mask1 = (self.events['height'][events['layer1'] & events['straightLineCutPulse']] == height1)
    mask2 = (self.events['height'][events['layer2'] & events['straightLineCutPulse']] == height2)
    mask3 = (self.events['height'][events['layer3'] & events['straightLineCutPulse']] == height3)

    self.events['straightPathL0Time'] = self.events['straightPathL0Time'][mask0]
    self.events['straightPathL1Time'] = self.events['straightPathL1Time'][mask1]
    self.events['straightPathL2Time'] = self.events['straightPathL2Time'][mask2]
    self.events['straightPathL3Time'] = self.events['straightPathL3Time'][mask3]

    self.events['straightPathDiffL03'] = self.events['straightPathL3Time'] - self.events['straightPathL0Time']
    print("testing", ak.drop_none(self.events['straightPathDiffL03']))

    '''output = self.events['straightPathL0Time']
    drop_empty = ak.num(output) > 0
    print("output", ak.drop_none(output[drop_empty]))'''


In [107]:
@mqCut
def centralTime(self):
    events = self.events
    timeCut = (events['timeFit_module_calibrated'] > 1100) & (events['timeFit_module_calibrated'] < 1400)
    drop_empty = ak.num(timeCut) > 0
    #newEvents = events[timeCut]
    newEvents = drop_empty & timeCut
    #print()
    #self.events = events[newEvents]
    for branch in branches:
        #print(branch, type(branch))
        self.events[branch] = self.events[branch][newEvents]


In [108]:
@mqCut
def threeInLine(self):
    npeCut = 100

    for path in range(16):

        row = path//4
        col = path%4

        outputName = 'threeHitPath{}'.format(path)

        #print("Checking three in line for path {}, row {}, col {}, with area cut {}".format(path, row, col, areaCut))

        layer0 = (self.events.layer0) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer1 = (self.events.layer1) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer2 = (self.events.layer2) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer3 = (self.events.layer3) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)

        threeStraight_f0 = ak.any(layer1, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f1 = ak.any(layer0, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f2 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f3 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer2, axis=1)

        self.events[outputName+'_f0'] = threeStraight_f0
        self.events[outputName+'_f1'] = threeStraight_f1
        self.events[outputName+'_f2'] = threeStraight_f2
        self.events[outputName+'_f3'] = threeStraight_f3

        self.events[outputName+'_s0'] = (threeStraight_f0) & (self.events.layer0) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s1'] = (threeStraight_f1) & (self.events.layer1) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s2'] = (threeStraight_f2) & (self.events.layer2) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s3'] = (threeStraight_f3) & (self.events.layer3) & (self.events.nPE >= npeCut)
        

@mqCut
def firstPulseCut(self, cutName='firstPulse', cut=False, branches=None):

    self.events[cutName] = self.events.ipulse == 0

    if cut:
        for branch in branches:
            self.events[branch] = self.events[branch][self.events[cutName]]

In [94]:
'''@mqCut
def heightCutMod(self, cutName='heightCut', heightCut=800, cut=True, branches=None):
    self.events[cutName] = self.events['height'] >= int(heightCut)
    print(self.events[cutName])
    print(self.events['height'][self.events[cutName]])
    if cut:
        print("In cut", branches)
        for branch in branches:
            print("branch", branch)
            self.events[branch] = self.events[branch][self.events[cutName]]'''

'@mqCut\ndef heightCutMod(self, cutName=\'heightCut\', heightCut=800, cut=True, branches=None):\n    self.events[cutName] = self.events[\'height\'] >= int(heightCut)\n    print(self.events[cutName])\n    print(self.events[\'height\'][self.events[cutName]])\n    if cut:\n        print("In cut", branches)\n        for branch in branches:\n            print("branch", branch)\n            self.events[branch] = self.events[branch][self.events[cutName]]'

need to figure out how to select the max pulse in the time window only

In [95]:
files = []

dir = '../skim/'
for filename in os.listdir(dir):
    if "skim" in filename and filename.endswith('.root'):
        files.append(dir+filename)

files

['../skim/MilliQan_Run1100_v34_skim.root',
 '../skim/MilliQan_Run1300_v34_skim.root',
 '../skim/MilliQan_Run1510_v34_skim.root',
 '../skim/MilliQan_Run1500_v34_skim.root',
 '../skim/MilliQan_Run1000_v34_skim.root',
 '../skim/MilliQan_Run1200_v34_skim.root',
 '../skim/MilliQan_Run1400_v34_skim.root']

In [115]:
#define a file list to run over
filelist = ['MilliQan_Run1000_v34_skim_correction.root']

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag', 'fileNumber', 'runNumber', 'type', 'ipulse', 'nPE',
            'time_module_calibrated', 'timeFit_module_calibrated', 'timeFit_module_calibrated_corrected', 'row', 'column', 'layer', 'height', 'area']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#height cut to get large pulses
muonHeightCut = mycuts.getCut(mycuts.heightCut, 'muonHeightCut', heightCut=1200, cut=True, branches=branches)

#muon area cut
muonAreaCut = getCutMod(mycuts.areaCut, mycuts, 'muonAreaCut', areaCut=300000, cut=True, branches=branches)

#muonAreaCut = mycuts.areaCut('muonAreaCut, areaCut=500000', cut=True, branches=branches)
#muonAreaCut = mycuts.getCut(mycuts.areaCut, 'muonAreaCut', areaCut=500000, cut=True, branches=branches)

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
bins = 400
xmin = 1100
xmax = 1500

h_pulseTime03 = r.TH2F("h_pulseTime03", "Pulse Times Between Layer 0 and 3", bins, xmin, xmax, bins, xmin, xmax)
h_L0Times = r.TH1F('h_L0Times', "Pulse Times Layer 0", 400, 1100, 1500)
h_L3Times = r.TH1F('h_L3Times', "Pulse Times Layer 3", 400, 1100, 1500)
h_TimeDiff = r.TH1F('h_TimeDiff', "Difference in Layer 0 and 3 Times", 100, -50, 50)
h_height = r.TH1F('h_height', "Height of Passing Pulses", 650, 0, 1300)
h_area = r.TH1F('h_area', "Area of Passing Pulses", 1000, 0, 100e4)

setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "pulseTime", pulseTime)
setattr(milliqanCuts, 'firstPulseCut', firstPulseCut)

'''setattr(milliqanCuts, "pulseTime", pulseTime)
setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "muonHeightCut", muonHeightCut)
#setattr(milliqanCuts, "muonAreaCut", muonAreaCut)'''

firstPulse = getCutMod(mycuts.firstPulseCut, mycuts, 'firstPulseCut', cutName='firstPulseCut', cut=True, branches=branches)


#add root histogram to plotter
myplotter.addHistograms(h_pulseTime03, ['straightPathL0Time', 'straightPathL3Time'])
myplotter.addHistograms(h_L0Times, 'straightPathL0Time')
myplotter.addHistograms(h_L3Times, 'straightPathL3Time')
myplotter.addHistograms(h_TimeDiff, 'straightPathDiffL03')
myplotter.addHistograms(h_height, 'height')
myplotter.addHistograms(h_area, 'area')

#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
cutflow = [boardMatchCut, pickupCut, mycuts.centralTime, firstPulse, muonAreaCut, mycuts.layerCut, mycuts.straightLineCut, 
            mycuts.pulseTime, myplotter.dict['h_pulseTime03'], myplotter.dict['h_height'], myplotter.dict['h_area'],
            myplotter.dict['h_L0Times'], myplotter.dict['h_L3Times'], myplotter.dict['h_TimeDiff']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=10000)

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("runs_1100_june3.root")

mycuts.getCutflowCounts()

----------------------------
MilliQan Scheduler:
	0. boardMatchCut
	1. pickupCut
	2. centralTime
	3. firstPulseCut
	4. muonAreaCut
	5. layerCut
	6. straightLineCut
	7. pulseTime
	8. h_pulseTime03
	9. h_height
	10. h_area
	11. h_L0Times
	12. h_L3Times
	13. h_TimeDiff
----------------------------
MilliQan Processor: Processing event 0...
testing [[1.86], [-0.358], [-2.35], [-0.577], ..., [-21.1], [-15.3], [-11.9], [-15.9]]
Number of processed events 2945
----------------------------------Cutflow Table------------------------------------
Cut                       N Passing Events     N Passing Pulses
-----------------------------------------------------------------------------------
boardsMatched             2944                 105458    
pickupCut                 2944                 69760     
centralTime               2943                 38753     
firstPulseCut             2942                 38119     
muonAreaCut               2938                 14699     
layerCut             

Warning in <TROOT::Append>: Replacing existing TH1: h_pulseTime03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_L0Times (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_L3Times (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_TimeDiff (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_height (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_area (Potential memory leak).


In [54]:
print('pulseTime' in locals())

True


In [98]:
c1 = r.TCanvas("c1", "c1", 600, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [116]:
h_pulseTime03.SetMarkerSize(10)
h_pulseTime03.SetMarkerStyle(7)
h_pulseTime03.Draw("")
line0 = r.TLine(1200, 1200, 1400, 1400)
line40p = r.TLine(1240, 1200, 1415, 1375)
line40m = r.TLine(1160, 1200, 1360, 1400)
line0.Draw("same")
line40p.Draw("same")
line40m.Draw("same")
c1.Draw()

In [78]:
h_height.Draw()
c1.Draw()

In [79]:
h_area.Draw()
c1.Draw()

In [36]:
h_L0Times.Draw()
c1.Draw()

In [37]:
h_L3Times.Draw()
c1.Draw()

In [100]:
h_TimeDiff.Rebin(2)
h_TimeDiff.Draw()
c1.Draw()

Plot the beam on/off files together

In [31]:
fin_beamOn = r.TFile.Open('layerTimes_april29_timeFit_On.root')
fin_beamOff = r.TFile.Open('layerTimes_april26_timeFit_Off.root')

In [32]:
#get all relevant plots
fin_beamOn.ls()

h_PulseTime_on = fin_beamOn.Get('h_pulseTime03')
h_PulseTime_off = fin_beamOff.Get('h_pulseTime03')

h_timeDiff_on = fin_beamOn.Get('h_TimeDiff')
h_timeDiff_off = fin_beamOff.Get('h_TimeDiff')

TFile**		layerTimes_april29_timeFit_On.root	
 TFile*		layerTimes_april29_timeFit_On.root	
  KEY: TH2F	h_pulseTime03;3	Pulse Times Between Layer 0 and 3 [current cycle]
  KEY: TH2F	h_pulseTime03;2	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH2F	h_pulseTime03;1	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH1F	h_L0Times;3	Pulse Times Layer 0 [current cycle]
  KEY: TH1F	h_L0Times;2	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L0Times;1	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L3Times;3	Pulse Times Layer 3 [current cycle]
  KEY: TH1F	h_L3Times;2	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_L3Times;1	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_TimeDiff;3	Difference in Layer 0 and 3 Times [current cycle]
  KEY: TH1F	h_TimeDiff;2	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_TimeDiff;1	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_height;3	Height of Passing Pulses [current cycle]
  KEY: TH1F	h_height;2	Height of Pa

In [33]:
h_PulseTime_on.SetMarkerStyle(7)
h_PulseTime_on.Draw()
h_PulseTime_off.SetMarkerColor(2)
h_PulseTime_off.SetMarkerStyle(7)
h_PulseTime_off.Draw("same")
c1.Draw()

In [35]:
h_timeDiff_off.Rebin(2)
h_timeDiff_on.Rebin(2)
h_timeDiff_off.SetLineColor(2)
h_timeDiff_on.Draw()

h_timeDiff_off.Draw("same")
c1.Draw()